In [ ]:
!pip install -q --upgrade transformers accelerate ipython-autotime

# import modules

In [1]:
from transformers import pipeline , AutomaticSpeechRecognitionPipeline , WhisperTokenizer, WhisperTimeStampLogitsProcessor ,WhisperForConditionalGeneration , WhisperProcessor ,AutoModelForSpeechSeq2Seq, AutoModelForCausalLM, AutoProcessor
import torch 

In [2]:
%load_ext autotime

time: 0 ns (started: 2024-11-29 10:59:42 +05:30)


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(device,torch_dtype)

cuda:0 torch.float16
time: 31 ms (started: 2024-11-29 10:59:42 +05:30)


In [ ]:
#input model
model_name = "openai/whisper-large-v3"
language = "English"
language_abbr = "en"
task= 'transcribe'

time: 0 ns (started: 2024-11-29 12:37:40 +05:30)


In [ ]:
whisper_asr = pipeline(
    "Automatic-Speech-Recognition",
    model = model_name,
    chunk_length_s = 30,
    device = device 
)

In [ ]:
def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = seconds % 60
    return (f"{hours:02}:{minutes:02}:{seconds:06.3f}").replace(".", ",")

time: 0 ns (started: 2024-11-29 11:03:03 +05:30)


In [ ]:
import os

def process_audio_and_create_vtt(audiofile_path, audio_type, model, outputfilename=None):
    # Transcribe the audio file with timestamps
    prediction = model(audiofile_path, return_timestamps=True)
    
    # Determine output VTT file name
    audiofile_name = os.path.splitext(os.path.basename(audiofile_path))[0]  # Extract file name without extension
    vtt_file_name = outputfilename if outputfilename else f"{audiofile_name}.vtt"

    # Write the transcription to the VTT file
    with open(vtt_file_name, "w", encoding="utf-8") as f:
        f.write("WEBVTT\n\n")
        for chunk in prediction["chunks"]:
            start_time = format_time(chunk["start"])
            end_time = format_time(chunk["end"])
            text = chunk["text"]
            f.write(f"{start_time} --> {end_time}\n{text}\n\n")


def process_directory_audio_files(directory_path, model):
    """
    Process all .mp3 files in a directory and create corresponding VTT files.

    Parameters:
    - directory_path: Path to the directory containing audio files
    - model: The transcription model to use
    """
    # Get all .mp3 files in the directory
    audio_files = [f for f in os.listdir(directory_path) if f.endswith(".mp3")]
    
    if not audio_files:
        print("No .mp3 files found in the directory.")
        return

    # Process each audio file
    for audio_file in audio_files:
        audiofile_path = os.path.join(directory_path, audio_file)
        output_file = os.path.join(directory_path, f"{os.path.splitext(audio_file)[0]}.vtt")
        
        print(f"Processing: {audio_file}")
        process_audio_and_create_vtt(audiofile_path, "mp3", model, output_file)

    print("All .mp3 files in the directory have been processed.")



